In [ ]:
import utils
import time
import numpy as np
%matplotlib inline

The util functions work with numpy arrays. The starting numpy array to work with is the architecture of the device.

In [ ]:
%%time
a = utils.chipresources()
print(a.shape)

In order to place LUTs on each slice, you have to give that slice a number. You can also plot the supposed placement. If there are areas that you want to forbid placement, you give them -1. 

In [ ]:
%%time
a[40:60, 20:30] = 4
a[40:60, 30:40] = 1
a[60:65, 25:35] = -1
utils.plot_layout(a)

Normally you would want to perform our experiments on certain types of slices (slice-L). There is a function that gets your pacement and proposes another one that satisfies the slice type condition (and forbidden slice condition).

In [ ]:
%%time
b = utils.check_and_propose(a, slice_type='L')

In [ ]:
%%time
b = utils.check_and_propose(a, slice_type='M') #   takes longer as there are fewer slicesMs

Now lets produce location constraints for the RO IP as per the design in:
https://github.com/sarashs/ring_oscillator_zynq

Note that the number of LUT per ROs in the input numpy array is one more than the number of stages (inverters). The extra LUT is a pass LUT that acts as a delay.

In [ ]:
# prepare the lut locations (upto 4 per slice) for a set of 32, 3 stage ROs
Num_Oscillators = 32
Num_Stages = 3# 3 stages
RO_locations = utils.chipresources() # reading the architecture file
for i in range(7, 75, 18): # setting the locations of 32 ROs, 128 LUTs in total
    for j in range(57, 93, 10):
        RO_locations[j, i] = 4
for i in range(125, 144, 6): 
    for j in range(57, 93, 10):
        RO_locations[j, i] = 4
utils.plot_layout(RO_locations)
# sets the placement constraints and satisfies the slice conditions
RO_locations = utils.RO_xdc(Num_Oscillators, Num_Stages, RO_locations, outputfile='ROs.XDC', slice_type='L')

Now we prepare the constraint files for the BTI sensors

In [ ]:
Num_Oscillators = 31
BTI_locations = np.copy(RO_locations)
BTI_locations[BTI_locations != 0] = -1 # slices that were used for ROs become forbidden (-1) 
for i in range(7, 75, 18): # setting the locations of 31 BTIs, 93 LUTs in total
    for j in range(58, 93, 10):
        BTI_locations[j, i] = 3
for i in range(125, 144, 6): 
    for j in range(58, 93, 10):
        BTI_locations[j, i] = 3
BTI_locations[j, i] = 0 # remember this time we only have 31
BTI_locations = utils.BTI_xdc(Num_Oscillators, BTI_locations, slice_type='L')

In [ ]:
sensor_locations = RO_locations + BTI_locations
utils.plot_layout(sensor_locations)

The last item to add is the heating elements. We have a function `fill` that can help with it.

In [ ]:
Num_Blocks = 64 # per IP number of blocks of heater
Block_Size = 36 # number of SHE (LUT) per block
heater_locations = np.copy(sensor_locations)
heater_locations[heater_locations != 0] = -1
heater_locations = utils.fill(heater_locations, init_coord=(55, 123), dim=(24, 24))
heater_locations = utils.heater_xdc(heater_locations, Num_Blocks, Block_Size)

In [ ]:
everything = heater_locations + sensor_locations
utils.plot_layout(everything)